In [23]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import f1_score
from sklearn.model_selection import StratifiedKFold,KFold
from sklearn.metrics import mean_absolute_error
import multiprocessing
from sklearn.metrics import f1_score
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

In [24]:
df=pd.read_csv('data/train.csv',header=0,names=['date','AQI','qua','PM2.5','PM10','SO2','CO','NO2','O3_8h','IPRC'])
df2=pd.read_csv('data/test.csv',header=0,names=['date','AQI','qua','PM2.5','PM10','SO2','CO','NO2','O3_8h'])
df

,date,AQI,qua,PM2.5,PM10,SO2,CO,NO2,O3_8h,IPRC
0,2016/1/1,293,重度污染,243,324,122,6.1,149,12,2.088378
1,2016/1/2,430,严重污染,395,517,138,7.5,180,18,3.316942
2,2016/1/3,332,严重污染,282,405,72,6.3,130,10,2.516425
3,2016/1/4,204,重度污染,154,237,73,3.5,72,34,1.505693
4,2016/1/5,169,中度污染,128,186,99,3.2,66,39,1.210233
...,...,...,...,...,...,...,...,...,...,...
361,2016/12/27,147,轻度污染,112,131,33,1.8,60,30,0.889445
362,2016/12/28,204,重度污染,154,181,33,1.9,65,29,1.346602
363,2016/12/29,328,严重污染,278,325,71,4.5,96,13,2.305147
364,2016/12/30,279,重度污染,229,273,60,3.7,92,11,1.872744


In [25]:
def get_fea(df):
    df['date'] = pd.to_datetime(df['date'], format='%Y/%m/%d')
    df["month"]=df["date"].apply(lambda x : x.month)
    df["day"]=df["date"].apply(lambda x : x.day)
    return df

In [26]:
train = get_fea(df)
test = get_fea(df2)
for col in ['qua']:
    le = LabelEncoder()
    train[col] = le.fit_transform(train[col])
    test[col] = le.transform(test[col])

In [27]:
score=[]
answers=[]
fea = [f for f in train.columns if f not in ['date','IPRC']]
# model = lgb.LGBMRegressor(max_depth=15,num_leaves=35,learning_rate=0.03,n_estimators=5000,seed=2020)
model = xgb.XGBRegressor(max_depth=7,learning_rate=0.05,n_estimators=10000,subsample=0.8)

n_fold = 5
folds = KFold(n_splits=n_fold, shuffle=True,random_state=2021)
for fold_n, (train_index, valid_index) in enumerate(folds.split(train)):
    X_train, X_valid = train[fea].iloc[train_index], train[fea].iloc[valid_index]
    y_train, y_valid = train['IPRC'][train_index], train['IPRC'][valid_index]
    
    model.fit(X_train,y_train,eval_set=[(X_valid, y_valid)],verbose=0,early_stopping_rounds=100)
    y_pre=model.predict(X_valid)
    print('每一折验证分数:'+str(mean_absolute_error(y_valid,y_pre)))
    score = score + mean_squared_error(y_valid,y_pre)
    y_pred_valid = model.predict(test[fea])
    answers.append(y_pred_valid)

每一折验证分数:0.044343326294809096
每一折验证分数:0.020716729656543257
每一折验证分数:0.02215274308380274
每一折验证分数:0.02052453325417224
每一折验证分数:0.022610665214254096


In [28]:
df2=pd.read_csv('data/test.csv',header=0,names=['date','AQI','qua','PM2.5','PM10','SO2','CO','NO2','O3_8h'])
df2['IPRC']=sum(answers)/n_fold
df2[['date','IPRC']].to_csv('sub4.csv',index=False)